# Test Pipeline of "Convert Labeled Data to Image Directory" Module

Use this pipeline to test labeled dataset in image classification sample.

In [ ]:
import os
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, Pipeline, dsl

## Configure workspace and compute

In [ ]:
# configure workspace information here.
workspace = Workspace.get(
    name='heta-EUS',
    subscription_id='e9b2ec51-5c94-4fa8-809a-dc1e695e4896',
    resource_group='thy-experiment'
)
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

## Prepare dataset
This is a labeled dataset generated with "Data Labeling".

In [ ]:
from azureml.core.dataset import Dataset

labeled_dataset = Dataset.get_by_name(workspace, name='duck_frog_20200713_042225').as_named_input('input_data')

## Load modules

In [ ]:
# load the registered custom module to convert labeled data to image directory.
convert_func = Module.load(workspace, namespace='heta-EUS', name='Convert Labeled Data to Image Directory')

In [ ]:
# load built-in modules in image sample.
init_image_transformation_func, densenet_func, score_image_model_func, apply_image_transformation_func, train_pytorch_model_func, convert_to_image_directory_func = Module.batch_load(workspace, identifiers=[('Init Image Transformation', 'azureml'), ('DenseNet', 'azureml'), ('Score Image Model', 'azureml'), ('Apply Image Transformation', 'azureml'), ('Train PyTorch Model', 'azureml'), ('Convert to Image Directory', 'azureml')])

## Set up a test pipeline

In [ ]:
# define pipeline
@dsl.pipeline(name='test data labeling', description='test data labeling', default_compute_target='cpu-d13v2')
def generated_pipeline():
    convert_labeled_to_image_directory_0 = convert_func(
        labeled_dataset=labeled_dataset
    )

    init_image_transformation_0 = init_image_transformation_func(
        resize='True',
        size=256,
        center_crop='True',
        crop_size=224,
        pad='False',
        color_jitter=False,
        grayscale=False,
        random_resized_crop='False',
        random_crop='False',
        random_horizontal_flip=True,
        random_vertical_flip=False,
        random_rotation='False',
        random_affine='False',
        random_grayscale=False,
        random_perspective=False)
    
    apply_image_transformation_0 = apply_image_transformation_func(
        input_image_directory=convert_labeled_to_image_directory_0.outputs.output_image_dir,
        input_image_transformation=init_image_transformation_0.outputs.output_image_transformation,
        mode='For training')
    
    densenet_0 = densenet_func(
        model_name='densenet201',
        pretrained=True,
        memory_efficient=False)
    
    train_pytorch_model_0 = train_pytorch_model_func(
        untrained_model=densenet_0.outputs.untrained_model,
        training_dataset=apply_image_transformation_0.outputs.output_image_directory,
        validation_dataset=apply_image_transformation_0.outputs.output_image_directory,
        epochs=5,
        batch_size=16,
        learning_rate=0.001,
        random_seed=1,
        patience=3)
    
    score_image_model_0 = score_image_model_func(
        trained_model=train_pytorch_model_0.outputs.trained_model,
        dataset=apply_image_transformation_0.outputs.output_image_directory)

In [ ]:
# create a pipeline
pipeline = generated_pipeline()

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
# submit a pipeline run
pipeline.submit(experiment_name='data_labeling').wait_for_completion()